In [7]:
from html import escape

In [2]:
def html_escape(arg):
    return escape(str(arg))

In [3]:
def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

In [4]:
def html_real(a):
    return '{0:.2f}'.format(round(a, 2))

In [5]:
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

In [7]:
def html_list(l):
    items = ['<li>{0}</li>'.format(html_escape(item)) for item in l]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [8]:
def html_dict(d):
    items = ['<li>{0}={1}</li>'.format(html_escape(k), html_escape(v)) for k, v in d.items()]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'


In [9]:
print(html_str('''this is a
multiline
string'''))

this is a<br/>
multiline<br/>
string


In [10]:
print(html_int(255))

255(<i>0xff</i>)


In [11]:
print(html_escape(3+10j))

(3+10j)


In [2]:
from decimal import Decimal

In [3]:
def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, (float, Decimal)):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, (list, tuple)):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    else:
        return html_escape(arg)
        

In [14]:
print(htmlize(10))

10(<i>0xa</i>)


In [15]:
print(htmlize(3.14159))

3.14


In [16]:
print(htmlize('this is a string'))

this is a string


In [17]:
print(htmlize([1, 2, 3]))

<ul>
<li>1</li>
<li>2</li>
<li>3</li>
</ul>


In [18]:
print(htmlize({'a': 1, 'b': 2, 'c': 3}))

<ul>
<li>a=1</li>
<li>b=2</li>
<li>c=3</li>
</ul>


In [20]:
print(htmlize(["""Python Rocks 
               skdks
               """, (10, 320), 300]))

<ul>
<li>Python Rocks 
               skdks
               </li>
<li>(10, 320)</li>
<li>300</li>
</ul>


In [17]:
def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, (float, Decimal)):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, (list, tuple)):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    elif isinstance(arg, set):
        return html_set(arg)
    else:
        return html_escape(arg)
        

In [18]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

def html_real(a):
    return '{0:.2f}'.format(round(a, 2))

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = ['<li>{0}</li>'.format(htmlize(item)) for item in l]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'
    
def html_dict(d):
    items = ['<li>{0}={1}</li>'.format(html_escape(k), htmlize(v)) for k, v in d.items()]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_set(arg):
    return html_list(arg)


In [14]:
print(htmlize(10))

10(<i>0xa</i>)


In [16]:
print(htmlize(["""Python Rocks
               skdks
               """, (10, 320), 300]))

<ul>
<li>Python Rocks<br/>
               skdks<br/>
               </li>
<li><ul>
<li>10(<i>0xa</i>)</li>
<li>320(<i>0x140</i>)</li>
</ul></li>
<li>300(<i>0x12c</i>)</li>
</ul>


In [19]:
print(htmlize({1, 2, 3}))

<ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
<li>3(<i>0x3</i>)</li>
</ul>


In [23]:
def htmlize(arg):
    registry = {
        int: html_int,
        object: html_escape,
        float: html_real,
        Decimal: html_real,
        str: html_str,
        list: html_list,
        tuple: html_list,
        dict: html_dict,
        set: html_set
    }
    fn = registry.get(type(arg), registry[object])
    return fn(arg)


In [27]:
def single_dispatch(fn):
    registry = {}
    registry[object] = fn
    def decorated(arg):
        return registry[object](arg)
    return decorated

In [28]:
@single_dispatch
def htmlize(arg):
    return html_escape(arg)

In [29]:
htmlize('1 < 100')

'1 &lt; 100'

In [30]:
def single_dispatch(fn):
    registry = {}
    registry[object] = fn
    registry[int] = lambda a: '{0}(<i>{1}</i>)'.format(a, str(hex(a)))
    registry[str] = lambda s: escape(s).replace('\n', '<br/>\n')
    def decorated(arg):
        return registry.get(type(arg), registry[object])(arg)
    return decorated

In [31]:
@single_dispatch
def htmlize(arg):
    return escape(str(arg))


In [32]:
htmlize(10)

'10(<i>0xa</i>)'

In [33]:
htmlize('1 < 100')

'1 &lt; 100'

In [72]:
def single_dispatch(fn):
    registry = {}
    registry[object] = fn
    def decorated(arg):
        return registry.get(type(arg), registry[object])(arg)
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
    
    def dispatch(type_):
        return registry.get(type_, registry[object])
    
    decorated.register = register
    decorated.dispatch = dispatch
    return decorated

In [73]:
@single_dispatch
def htmlize(arg):
    return escape(str(arg))

In [74]:
htmlize('1 < 10')

'1 &lt; 10'

In [75]:
htmlize(100)

'100'

In [76]:
htmlize

<function __main__.single_dispatch.<locals>.decorated(arg)>

In [77]:
htmlize.register

<function __main__.single_dispatch.<locals>.register(type_)>

In [82]:
@htmlize.register(int)
def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

In [83]:
htmlize(10)

'10(<i>0xa</i>)'

In [84]:
html_int

<function __main__.html_int(a)>

In [85]:
htmlize.register

<function __main__.single_dispatch.<locals>.register(type_)>

In [86]:
@htmlize.register(list)
@htmlize.register(tuple)
def html_sequence(l):
    items = ['<li>{0}</li>'.format(htmlize(item)) for item in l]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'


In [88]:
htmlize.dispatch(int)

<function __main__.html_int(a)>

In [89]:
print(htmlize([1, 2, 3]))

<ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
<li>3(<i>0x3</i>)</li>
</ul>


In [90]:
print(htmlize((1, 2, 3)))

<ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
<li>3(<i>0x3</i>)</li>
</ul>


In [91]:
htmlize.dispatch(list)

<function __main__.html_sequence(l)>

In [92]:
htmlize.dispatch(float)

<function __main__.htmlize(arg)>

In [93]:
htmlize(100)

'100(<i>0x64</i>)'

In [95]:
htmlize(True)

'True'

In [96]:
from numbers import Integral

In [97]:
isinstance(10, Integral)

True

In [98]:
class Person:
    pass


In [106]:
@single_dispatch
def htmlize(arg):
    return escape(str(arg))

In [107]:
@htmlize.register(Integral)
def html_integral(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))
    

In [108]:
isinstance(10, Integral)

True

In [109]:
htmlize(10)

'10'

In [110]:
htmlize.dispatch(Integral)

<function __main__.html_integral(a)>

In [115]:
@htmlize.register(int)
@htmlize.register(bool)
def html_integral_bool(a):
    return '{0}, {1}'.format(a, str(type(a)))

In [116]:
htmlize(True)

"True, <class 'bool'>"

In [117]:
from collections.abc import Sequence

In [118]:
@htmlize.register(Sequence)
def html_sequence(l):
    items = ['<li>{0}</li>'.format(htmlize(item)) for item in l]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [119]:
htmlize([1, 2, 3])

'[1, 2, 3]'

In [120]:
htmlize.dispatch(Sequence)

<function __main__.html_sequence(l)>

In [121]:
htmlize.dispatch(list)

<function __main__.htmlize(arg)>

In [1]:
from functools import singledispatch

In [2]:
from numbers import Integral
from collections.abc import Sequence

In [8]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [9]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>})

In [10]:
htmlize.dispatch(str)

<function __main__.htmlize(a)>

In [11]:
htmlize('Ten')

'Ten'

In [12]:
@htmlize.register(Integral)
def htmlize_int(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

In [13]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.htmlize_int(a)>})

In [14]:
htmlize.dispatch(int)

<function __main__.htmlize_int(a)>

In [15]:
htmlize(10)

'10(<i>0xa</i>)'

In [16]:
htmlize.dispatch(bool)

<function __main__.htmlize_int(a)>

In [17]:
htmlize(True)

'True(<i>0x1</i>)'

In [18]:
@htmlize.register(Sequence)
def html_sequence(l):
    items = ('<li>{0}</li>'.format(htmlize(item)) for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [20]:
print(htmlize([1, 2, 3]))

<ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
<li>3(<i>0x3</i>)</li>
</ul>


In [21]:
print(htmlize((1, 2, 3)))

<ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
<li>3(<i>0x3</i>)</li>
</ul>


In [22]:
print(htmlize('python'))

RecursionError: maximum recursion depth exceeded in comparison

In [25]:
@htmlize.register(str)
def html_str(s):
    return escape(s).replace('\n', '<br/>\n')

In [26]:
htmlize('python is great')

'python is great'

In [27]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.htmlize_int(a)>,
              collections.abc.Sequence: <function __main__.html_sequence(l)>,
              str: <function __main__.html_str(s)>})

In [28]:
print(htmlize((1,2,3)))

<ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
<li>3(<i>0x3</i>)</li>
</ul>


In [29]:
print(htmlize([1,2,3]))

<ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
<li>3(<i>0x3</i>)</li>
</ul>


In [33]:
@htmlize.register(tuple)
def html_tuple(t):
    items = (escape(str(item)) for item in t)
    return '{0}'.format(','.join(items))

In [34]:
print(htmlize((1,2,3)))

1,2,3


In [35]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.htmlize_int(a)>,
              collections.abc.Sequence: <function __main__.html_sequence(l)>,
              str: <function __main__.html_str(s)>,
              tuple: <function __main__.html_tuple(t)>})

In [36]:
@htmlize.register(Integral)
def _(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

@htmlize.register(Sequence)
def _(l):
    items = ('<li>{0}</li>'.format(htmlize(item)) for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

@htmlize.register(str)
def _(s):
    return escape(s).replace('\n', '<br/>\n')

In [37]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__._(a)>,
              collections.abc.Sequence: <function __main__._(l)>,
              str: <function __main__._(s)>,
              tuple: <function __main__.html_tuple(t)>})

In [38]:
print(htmlize(100))

100(<i>0x64</i>)


In [39]:
print(htmlize([1,2,3]))

<ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
<li>3(<i>0x3</i>)</li>
</ul>


In [40]:
print(htmlize('Python'))

Python


In [41]:
htmlize.dispatch(Integral)

<function __main__._(a)>

In [42]:
htmlize.dispatch(str)

<function __main__._(s)>

In [43]:
id(htmlize.dispatch(Integral))

140400617533792

In [44]:
id(htmlize.dispatch(str))

140400617535520

In [45]:
id(_)

140400617535520